## Importing libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from scipy.stats import moment

# Statsmodels
import statsmodels

# Pytorch
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

from sklearn.metrics import mean_squared_error
import plotly.express as px

import time
from matplotlib import pyplot
from sklearn.preprocessing import MinMaxScaler

import joypy
import time
from matplotlib import pyplot
from sklearn.preprocessing import MinMaxScaler
from transformer.model import TransformerTimeSeries
from transformer.utils import TransformerSet, batch_train, batch_val
import os
import random

## Get train and test sets

In [2]:
train_set = pd.read_csv('train.csv')
test_set = pd.read_csv('test.csv')
train_set['date'] = pd.to_datetime(train_set['date'], format="%Y-%m-%d")

In [3]:
train_set.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


## Insert the notion of weekdays  and days of month

In [4]:
train_set['year']    =  train_set['date'].dt.year
train_set['month']   =  train_set['date'].dt.month
train_set['day']     =  train_set['date'].dt.dayofyear
train_set['weekday'] =  train_set['date'].dt.weekday
train_set['weekend'] =  train_set['weekday'] > 4

## Verifing the correlation between weekendays and sales

In [5]:
print(train_set['weekday'].max())
print(train_set['weekday'].min())
bar_df = {"weekday" : [], 
          "mean": []}
for value in range(train_set['weekday'].min(),train_set['weekday'].max() + 1):
    bar_df[f"weekday"].append(value)
    bar_df[f"mean"].append(train_set[(train_set['weekday'] == value)]['sales'].mean()) 

fig = px.bar(bar_df, x='weekday', y='mean', color='weekday',
             labels={'Mean value per weekday'}, height=400)
fig.show()

6
0


### This shows that the sales value increases in the weekends and decreces in the weekdays
This means that the weekdays could be valuable income data in a machine learning model, this is possible to know beforehand what the weekdays of the desirable prediction and this data presents a direct correlation between the value of the sales. Thus, the `weekdays` value could be a good entry feature of the model.

## Geting the series of one store and one item

In [6]:
item_set = train_set[(train_set['item'] == 1) & (train_set['store'] == 1)]

In [7]:
item_train_set = item_set[(item_set['date'] < "2016-12-31")]
item_test_set  = item_set[(item_set['date'] > "2016-12-31")]

In [8]:
item_train_set.head()

,date,store,item,sales,year,month,day,weekday,weekend
0,2013-01-01,1,1,13,2013,1,1,1,False
1,2013-01-02,1,1,11,2013,1,2,2,False
2,2013-01-03,1,1,14,2013,1,3,3,False
3,2013-01-04,1,1,13,2013,1,4,4,False
4,2013-01-05,1,1,10,2013,1,5,5,True


In [9]:
item_test_set.head()

,date,store,item,sales,year,month,day,weekday,weekend
1461,2017-01-01,1,1,19,2017,1,1,6,True
1462,2017-01-02,1,1,15,2017,1,2,0,False
1463,2017-01-03,1,1,10,2017,1,3,1,False
1464,2017-01-04,1,1,16,2017,1,4,2,False
1465,2017-01-05,1,1,14,2017,1,5,3,False


## The data behavior

In [10]:
fig = px.line(item_train_set, x='date', y='sales')
fig.show()

## Data Autocorrelation plot and analysis

By selecting only the first month in the correlation plot we can visualize that our sales series presents a weekly peak, this is a strong indicator that the sales behavior have a weekly period. The same can be visualized by selecting a year of the sales data, this show us that this data have a annual period too.

In [11]:
item_train_set['auto_corr'] = statsmodels.tsa.stattools.acf(item_train_set['sales'].values, nlags = len(item_train_set), fft=False)
fig = px.line(item_train_set, x='date', y='auto_corr')
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="todate"),
            dict(count=6, label="6m", step="month", stepmode="todate"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="todate"),
            dict(step="all")
        ])
    )
)
fig.show()


<ipython-input-11-df643327bc43>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Stationary test
One of the most important analyses of time series data is to detect if the subject series is stationary or not, if not further transformations are required to remove trend and seasonality of the series, this analysis is even more important using statistic based models. Many authors of papers that use neural networks (NN) for time series defend that using this kind of machine learning model does not necessarily require the data to be already stationary. Although, is important to know if the data is stationary even if the model selected is a NN, because if the model does not converges well this could be a cause and know the properties of the series is highly valuable as well.

[1] Define a weak stationary time series if the mean function $ E[x(t)] $ is independent of $ t $, if the autocoraviation function $Cov (x(t+h), x(t))$ is independent of $ t $ for each $h$ and if $E[x^2[n]]$ is finite for each $n$.

To perform the weak stationary test, the mean function and the autocovariation function ware applied over rolling windows, since its sampled data. Thus, the window size has an impact over the functions interpretations, the window represents the interval in which the stationary hypothesis is tested.

Besides this definition, the $ statsmodels $ library has the Augmented Dickey-Fuller unit root test. The Augmented Dickey-Fuller test can be used to test for a unit root in a univariate process in the presence of serial correlation.

References
[1] Brockwell, Peter J., and Richard A. Davis. Introduction to time series and forecasting. springer, 2016.

In [12]:
def stationary_test(entry,delta=200,ad=False,std=False):
    window_size=int(len(entry)/15)
    # Weak stationary test
    # Mean function
    mean_y = []
    mean_x = []
    
    s_moment_y = []
    std_y = []
    n_data = len(entry)
    for i in range(0, int(n_data - window_size)):
        # Roling window start and end
        n_start = i
        n_end = n_start + window_size
        # Mean, standard deviation and second moment calculation
        mean_y_i = np.mean(entry[n_start:n_end])
        s_moment_y_i = moment(entry[n_start:n_end],moment=2)
        std_y_i = np.std(entry[n_start:n_end])
        # Saving the results 
        mean_y.append(mean_y_i)
        mean_x.append(n_end)
        s_moment_y.append(s_moment_y_i)
        std_y.append(std_y_i)

    # Autocovariance function
    acov_y = []
    acov_x = []
    n_data = len(entry)
    for i in range(0, int(n_data - window_size - delta)):
        n_start = i
        n_end = n_start + window_size
        acov_y_i = np.cov(
            entry[n_start:n_end], entry[n_start+delta:n_end+delta]
        )[0][0]
        acov_y.append(acov_y_i)
        acov_x.append(n_end)
    if(ad):
        result = adfuller(entry)
        print("ADF Statistic: %f" % result[0])
        print("p-value: {0}".format(result[1]))
        print("Critical Values:")
        for key, value in result[4].items():
            print("\t%s: %.3f" % (key, value))
        # if the p-value < 0.05  and the adf statistic is less than
        # critical values the series is stationary or is time independent
        
    return [mean_x,mean_y],[acov_x,acov_y], s_moment_y, std_y

In [13]:
# Weak stationary test
sales_train = item_train_set['sales'].to_numpy()

mean, cov, s_moment, std = stationary_test(sales_train, delta=20,ad=True) 
mean_value = np.zeros(len(sales_train))
mean_value[mean[0][0]:] = mean[1]

cov_value = np.zeros(len(sales_train))
cov_value[cov[0][0]:len(cov[1])+cov[0][0]] = cov[1]

std_value = np.zeros(len(sales_train))
std_value[mean[0][0]:] = std

s_moment_value = np.zeros(len(sales_train))
s_moment_value[mean[0][0]:] = s_moment

item_train_set['mean'] = mean_value
item_train_set['cov'] = cov_value
item_train_set['s_moment'] = std_value

fig = px.line(item_train_set, x='date', y=['mean', 'cov', 'sales', 's_moment'])
fig.show()

ADF Statistic: -2.819719
p-value: 0.05551174940647831
Critical Values:
	1%: -3.435
	5%: -2.864
	10%: -2.568


<ipython-input-13-a1d872bacd3c>:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-13-a1d872bacd3c>:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-13-a1d872bacd3c>:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Analyzing the test results
The mean function test hypothesis failed, this can be noted by checking the function follows the trend and seasonlity of the sales series, the covariation function presents a linear behavior with a lot of disturbance, the standard deviation function seans to be linear like, but with a lot of noise. The second-moment function doesn't show a tendency to infinite on any $n$, that way passing on this condition.

Analyzing the functions plots it's correct to infer that this series of sales is non-stationary, this can be also noted on the autocorrelation plot, which shows a slow decaying behavior.the autocorrelation plot shows also a periodic behavior, which represents a periodic behavior on the time series, and also in several lags the ACF value goes above and bellow the confidence intervals of 99% and 95%

Besides the week sationary test, theAugmented Dickey-Fuller test can be done to check if the series of sales is stationary or not. This test checks if the series have a unit root and doing so it can make the assumption of how much the series is defined by it's trend.

There are 2 Hypothesis:

Null Hypothesis (H0): If failed to be rejected, it suggests the time series has a unit root, meaning it is non-stationary. It has some time dependent structure.

Alternate Hypothesis (H1): The null hypothesis is rejected; it suggests the time series does not have a unit root, meaning it is stationary. It does not have time-dependent structure.

We interpret this result using the p-value from the test. A p-value below a threshold (such as 5% or 1%) suggests we reject the null hypothesis (stationary), otherwise a p-value above the threshold suggests we fail to reject the null hypothesis (non-stationary).

p-value > 0.05: Fail to reject the null hypothesis (H0), the data has a unit root and is non-stationary.
p-value <= 0.05: Reject the null hypothesis (H0), the data does not have a unit root and is stationary.

Since the p-value of the test value applied to the series is bigger than 0.05, the null hypothesis is rejected, thus the series is non-stationary


## Using Neural networks for solving the time series problem
First, it is necessary to transform this time series problem into a supervised learning problem; for this, it is necessary to define whether the network will make a recursive forecast, in which the forecasts will be used as resources for a longer time horizon or if the network will make a direct forecast, in which the network only uses the last steps of time and resources known as the day of the week, to forecast several steps.

### Direct prediction approach
To predict the 3 months of sales is necessary to define first the labels and features of the model, as features the model will use 3 months of past sales data and the 3 months of weekdays value. As labels would be 3 months of the future data, to extrat the labels and features of the data a rolling window approach is reaquired, this way a window of 3 months past data, 3 months of weekdays value could be related to 3 months of future data

## Rolling window approach

With this approach is possible to turn a series of data into a supervised learning problem, this way the model can intepretate the problem properly.

In [14]:
# Scaling the entry data
sales_train   = item_train_set['sales'].to_numpy()
sales_train   = sales_train.reshape(len(sales_train),1)
scaler        = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(sales_train)
sales_train   = scaler.transform(sales_train).flatten()


index_val     = np.arange(len(sales_train))

# Three months of prediction
pred_horizon = 30
label_win = 30
feat_win  = 30
win_size  = label_win + feat_win
n_windows = len(sales_train) - win_size

In [15]:
def rolling_window(n_windows, feat_win, label_win, dataset):
    x_train = np.zeros((n_windows, int(feat_win + label_win)))
    x_id = np.zeros((n_windows, int(feat_win + label_win)))
    y_train = np.zeros((n_windows, feat_win + label_win))
    y_id = np.zeros((n_windows, feat_win + label_win))
    for win in range(n_windows):
        # Feature window start and end index
        i_s = win
        i_e = win + feat_win + label_win

        # Geting the past sales feature
        x_train[win][:feat_win] = dataset[i_s: i_e-label_win]
        x_id[win][:feat_win] = index_val[i_s: i_e-label_win]

        # Geting the weekdays feature
        # x_train[win][feat_win:] = weekday_train[i_s_label : i_e_label]

        # Since we want the weekdays to be corralated to the label values
        # the window of weekdays value that we will use will be the label's
        # weekday. This makes sense taking in count that we know the weekdays
        # beforehand

        # Geting the labels
        y_train[win] = dataset[i_s : i_e]
        y_id[win] = index_val[i_s : i_e]
    
    return x_train, x_id, y_train, y_id

In [16]:
x_train, x_id, y_train, y_id = rolling_window(n_windows, feat_win, label_win, sales_train)

In [17]:
y_id[0]

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38.,
       39., 40., 41., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51.,
       52., 53., 54., 55., 56., 57., 58., 59.])

In [18]:
x_id[0]

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [19]:
one_sample_df = {'x_train':x_train[0], 'y_train':y_train[0], 'index': np.arange(len(x_train[0]))}
fig = px.line(one_sample_df, x='index', y=['x_train', 'y_train'])
fig.show()
# plt.plot(x_train[0], label='features')
# plt.ylabel('Normalized value')
# plt.xlabel('Feature window index')
# plt.title("One sample of the model's features")
# plt.show()

# plt.plot(y_train[0], label='labels')
# plt.ylabel('Sales amount')
# plt.xlabel('label index')
# plt.title("One sample of the model's Labels")
# plt.show()


## Before define the model we should first define our random seeds to generate reproducible results

In [20]:
def seed_torch(seed=1029):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [21]:
# Select an arbitrary seed
seed_torch(42)

## Lets now create our test set

In [22]:
sales_test = item_test_set['sales'][:pred_horizon].to_numpy()
sales_test   = sales_test.reshape(len(sales_test),1)
sales_test   = scaler.transform(sales_test).flatten()
dates = item_test_set['date'][:label_win].to_numpy()

entry_test = np.zeros(int(feat_win+label_win))
entry_test[:feat_win] = sales_train[-feat_win:]

label_test = np.zeros(int(feat_win+label_win))
label_test[:feat_win] = sales_train[-feat_win:]

label_test[feat_win:] = sales_test[:label_win]

## Defining the model
The recommended neural network for a time series problem usually is a LSTM, in this example perhaps we will use the transformer network, as this model was said to have greater performance and lower computational cost than the traditional recurrent neural networks, such as LSTM or RNNS.

## Transformer network

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
model = TransformerTimeSeries(device, dropout=0.4, d_model=64, 
                              encoder_mask=True, 
                              nhead=4, 
                              encoder_only=True,
                              output_vector_sz=1,
                              iterative=False,
                              n_encoder_time_steps=feat_win+label_win,
                              n_input_time_steps=feat_win,
                              n_enc_layers=2
                             ).to(device)

criterion = nn.MSELoss()
lr = 0.005
batch_size= 60
epochs = 30
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.98)

Using device: cuda


In [24]:
x_train.shape[1]

60

In [25]:
train_set = TransformerSet(x_train, y_train, x_train.shape[1], encoder_only=True)
test_set  = TransformerSet(np.array([entry_test]),np.array([label_test]), x_train.shape[1], encoder_only=True)

train_loader = DataLoader(train_set,
                          batch_size=batch_size,
                          shuffle=True)
test_loader = DataLoader(test_set,
                          batch_size=batch_size,
                          shuffle=False)

In [26]:
loss = []
val_loss = []
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    epoch_loss = batch_train(model, epoch, batch_size, train_loader, criterion, optimizer, scheduler,  
                             y_train.shape[0], 
                             device, encoder_only=True,)
    val_epoch_loss = batch_val(model, test_loader, 
                               criterion, device, 
                               encoder_only=True,) 
    loss.append(epoch_loss)
    val_loss.append(val_epoch_loss)
    scheduler.step()

/home/rodrigo/home-hd/Anaconda/envs/pytorch/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:369: UserWarning:

To get the last learning rate computed by the scheduler, please use `get_last_lr()`.



| epoch   1 |     4/   23 batches | lr 0.005000 | 249.36 ms | loss 2.10748
| epoch   1 |     8/   23 batches | lr 0.005000 | 71.33 ms | loss 0.26567
| epoch   1 |    12/   23 batches | lr 0.005000 | 69.47 ms | loss 0.13304
| epoch   1 |    16/   23 batches | lr 0.005000 | 68.76 ms | loss 0.13725
| epoch   1 |    20/   23 batches | lr 0.005000 | 67.98 ms | loss 0.13155
| epoch   2 |     4/   23 batches | lr 0.004802 | 88.72 ms | loss 0.15559
| epoch   2 |     8/   23 batches | lr 0.004802 | 69.84 ms | loss 0.12278
| epoch   2 |    12/   23 batches | lr 0.004802 | 70.18 ms | loss 0.11699
| epoch   2 |    16/   23 batches | lr 0.004802 | 69.18 ms | loss 0.12330
| epoch   2 |    20/   23 batches | lr 0.004802 | 69.31 ms | loss 0.11984
| epoch   3 |     4/   23 batches | lr 0.004706 | 113.88 ms | loss 0.14659
| epoch   3 |     8/   23 batches | lr 0.004706 | 94.05 ms | loss 0.11846
| epoch   3 |    12/   23 batches | lr 0.004706 | 97.06 ms | loss 0.11331
| epoch   3 |    16/   23 batches | 

In [ ]:
plot_df ={'loss': loss,
          'val_loss': val_loss,
         'epochs': np.arange(len(loss))}

fig = px.line(plot_df, x='epochs', y=['loss', 'val_loss'])
fig.show()

In [ ]:
model.eval()
src, true_target = next(iter(test_loader))
src = Variable(torch.Tensor(src.float())).to(device)
output     = model(src).cpu().view(-1)
output = output.detach().numpy().reshape(len(output),1)
output = scaler.inverse_transform(output).flatten()

In [ ]:
label_test = true_target.cpu().view(-1).detach().numpy()
label_test = label_test.reshape(len(label_test),1)
label_test = scaler.inverse_transform(label_test).flatten()

In [ ]:
output.shape

(60,)

In [ ]:
plot_df ={'Date': dates,
          'pred': output[-pred_horizon:],
          'label': label_test[-pred_horizon:],
          'error': output[-pred_horizon:]-label_test[-pred_horizon:]}

fig = px.line(plot_df, x='Date', y=['pred','label','error'])
fig.show()

In [ ]:
mean_squared_error(label_test[-pred_horizon:], output[-pred_horizon:])

26.074062449362557

In [ ]:
attention_weights = model.encoder_attention(src, layer_idx=0)[1]
attention_weights = attention_weights.cpu().detach().numpy()
attention_weights = attention_weights.reshape(src.shape[1], src.shape[1])

In [ ]:
attention_weights = attention_weights[feat_win:, :feat_win]

In [ ]:
attention_weights.shape

(30, 30)

In [ ]:
fig = px.imshow(attention_weights, aspect="auto")
fig.show()